<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing CrewAI with Arize Phoenix - Prompt Chaining Workflow</h1>

In [ ]:
%pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

# Set up Keys and Dependencies

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)
*   Phoenix API key (https://app.phoenix.arize.com/)

In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass.getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

## Configure Tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="crewai-agents", endpoint="https://app.phoenix.arize.com/v1/traces"
)

# Instrument CrewAI

In [ ]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

## Define your Agents

In [ ]:
from crewai import Agent, Crew, Task
from crewai.process import Process

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="Research cutting-edge AI topics and summarize the top 3 trends.",
    backstory="Expert in AI research and trend analysis.",
    verbose=True,
)

content_strategist = Agent(
    role="Tech Content Strategist",
    goal="Create a structured article outline from the research.",
    backstory="Technical storyteller who crafts engaging outlines.",
    verbose=True,
)

content_reviewer = Agent(
    role="Content Reviewer",
    goal="Validate outline for clarity, tone, and completeness.",
    backstory="Editorial expert with a focus on technical accuracy.",
    verbose=True,
)

## Define your Tasks

In [ ]:
research_task = Task(
    description="Summarize the top 3 trends in open-source LLM development.",
    agent=research_analyst,
    expected_output="Bullet points of top 3 trends with brief explanations.",
)

outline_task = Task(
    description="Generate an article outline for CTOs based on the research.",
    agent=content_strategist,
    expected_output="Outline with title, sections, and key points.",
)

review_task = Task(
    description="Review the outline for quality and alignment.",
    agent=content_reviewer,
    expected_output="Reviewed outline with suggestions or approval.",
)

## Create Crew

In [ ]:
crew = Crew(
    agents=[research_analyst, content_strategist, content_reviewer],
    tasks=[research_task, outline_task, review_task],
    process=Process.sequential,
    verbose=True,
    full_output=True,
)

result = crew.kickoff()
print(result)

### Check your Phoenix project to view the traces and spans from your runs.